# Kaggle's starter code

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


# First Section

In this section, I chose a notebook on Kaggle to run and see how people preprocessed and trained the dataset.

You can get to this notebook [here](https://www.kaggle.com/code/ebrahimmerza/regression-of-used-car-price-lgbm-catboost-ensembl)

## Libraries

In [2]:
import re
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

## Data Loading

In [3]:
df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

In [4]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [5]:
df.shape

(188533, 13)

In [6]:
df_test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [7]:
df_test.shape

(125690, 12)

In [8]:
print("Info Train DF")
print(df.info())

print("----------")

print(df.isna().sum())

Info Train DF
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB
None
----------
id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmiss

In [9]:
print("Info Test DF")
print(df_test.info())

print("----------")

print(df_test.isna().sum())

Info Test DF
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB
None
----------
id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        3383
engine              0
transmission        0
ext_col             0
int_col  

We can that there are missing values in 3 columns: `fuel_type`, `clean_title`, `accident`

## Preprocessing

Fill N/A values in 3 columns: `fuel_type`, `clean_title`, `accident` with *Electricity*, *No*, *None reported*.

In [10]:
df['fuel_type'].fillna('Electricity', inplace=True)
df['clean_title'].fillna('No', inplace=True)
df['accident'].fillna('None reported', inplace=True)

In [11]:
df_test['fuel_type'].fillna('Electricity', inplace=True)
df_test['clean_title'].fillna('No', inplace=True)
df_test['accident'].fillna('None reported', inplace=True)

In [12]:
print(df.isna().sum())
print('---------------')
print(df_test.isna().sum())

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64
---------------
id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64


**Feature extraction from `engine`**

1 . Use regular express to extract from `engine` columns new features: `HP`, `displacement`, `engine_type`, `cylinder_count`, `fuel_type`.

In [13]:
def extract_hp(engine):
    match = re.search(r'(\d+(\.\d+)?)HP', engine)
    return float(match.group(1)) if match else None

def extract_displacement(engine):
    match = re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', engine)
    if match:
        # Check if the first group matched, otherwise use the second group
        return float(match.group(1) or match.group(2))

def extract_engine_type(engine):
    match = re.search(r'(V\d+|I\d+|Flat \d+|Straight \d+)', engine)
    return match.group(1) if match else None

def extract_cylinder_count(engine):
    match = re.search(r'(\d+) Cylinder', engine)
    return int(match.group(1)) if match else None

def extract_fuel_type(engine):
    fuel_types = ['Gasoline', 'Diesel', 'Electric', 'Hybrid', 'Flex Fuel']
    for fuel in fuel_types:
        if fuel in engine:
            return fuel
    return None

In [14]:
df['Horsepower'] = df['engine'].apply(extract_hp)
df['Displacement'] = df['engine'].apply(extract_displacement)
df['Engine Type'] = df['engine'].apply(extract_engine_type)
df['Cylinder Count'] = df['engine'].apply(extract_cylinder_count)
df['Fuel Type'] = df['engine'].apply(extract_fuel_type)

In [15]:
df_test['Horsepower'] = df_test['engine'].apply(extract_hp)
df_test['Displacement'] = df_test['engine'].apply(extract_displacement)
df_test['Engine Type'] = df_test['engine'].apply(extract_engine_type)
df_test['Cylinder Count'] = df_test['engine'].apply(extract_cylinder_count)
df_test['Fuel Type'] = df_test['engine'].apply(extract_fuel_type)

In [17]:
print(df.isna().sum())
print('---------------')
print(df_test.isna().sum())

id                    0
brand                 0
model                 0
model_year            0
milage                0
fuel_type             0
engine                0
transmission          0
ext_col               0
int_col               0
accident              0
clean_title           0
price                 0
Horsepower        33259
Displacement       6770
Engine Type       99563
Cylinder Count    37855
Fuel Type         31546
dtype: int64
---------------
id                    0
brand                 0
model                 0
model_year            0
milage                0
fuel_type             0
engine                0
transmission          0
ext_col               0
int_col               0
accident              0
clean_title           0
Horsepower        22181
Displacement       4438
Engine Type       66299
Cylinder Count    25150
Fuel Type         21015
dtype: int64


There are many empty/NaN values in three cols: `Horsepower`, `Displacement`, `Engine Type`, `Cyclinder Count`, `Fuel Type`. Now we use the KNNImputer to fill in N/A values in two columns: `Horsepower`, `Cyclinder Count`.

In [18]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[['Horsepower', 'Cylinder Count']] = imputer.fit_transform(df[['Horsepower', 'Cylinder Count']])
df_test[['Horsepower', 'Cylinder Count']] = imputer.transform(df_test[['Horsepower', 'Cylinder Count']])

2. Replace two common cases in columns `accident`, `clean_title` with 1 and 0.

In [19]:
df["accident"] = df["accident"].replace({'At least 1 accident or damage reported':1,"None reported":0}) 
df["accident"].value_counts()

accident
0    146966
1     41567
Name: count, dtype: int64

In [20]:
df_test["accident"] = df_test["accident"].replace({'At least 1 accident or damage reported':1,"None reported":0}) 
df_test["accident"].value_counts()

accident
0    97895
1    27795
Name: count, dtype: int64

In [21]:
df["clean_title"] = df["clean_title"].replace({"Yes":1,"No":0})
df["clean_title"].value_counts()

clean_title
1    167114
0     21419
Name: count, dtype: int64

In [22]:
df_test["clean_title"] = df["clean_title"].replace({"Yes":1,"No":0})
df_test["clean_title"].value_counts()

clean_title
1    111435
0     14255
Name: count, dtype: int64

In [23]:
df['Engine Type'].value_counts()

Engine Type
V6            57671
Straight 6    11640
V8             6944
I4             6004
Flat 6         5344
V12             405
V10             389
I6              317
I3              256
Name: count, dtype: int64

In [24]:
df['Engine Type'].value_counts().sum()

88970

Drop column `Engine Type` as it had few values. Drop two columns `engine` and `model` as they are extraced.

In [25]:
df.drop(['Engine Type'], axis=1, inplace=True)
df_test.drop(['Engine Type'], axis=1, inplace=True)

In [26]:
columns_to_drop = ['engine', 'model']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
df_test.drop(columns=[col for col in columns_to_drop if col in df_test.columns], inplace=True)

Here's the new DataFrame.

In [27]:
df.head()

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,Fuel Type
0,0,MINI,2007,213000,Gasoline,A/T,Yellow,Gray,0,1,4200,172.0,1.6,4.0,Gasoline
1,1,Lincoln,2002,143250,Gasoline,A/T,Silver,Beige,1,1,4999,252.0,3.9,8.0,Gasoline
2,2,Chevrolet,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,0,1,13900,320.0,5.3,8.0,Flex Fuel
3,3,Genesis,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,0,1,45000,420.0,5.0,8.0,Gasoline
4,4,Mercedes-Benz,2021,7388,Gasoline,7-Speed A/T,Black,Beige,0,1,97500,208.0,2.0,4.0,Gasoline


In [28]:
df_test.head()

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,Horsepower,Displacement,Cylinder Count,Fuel Type
0,188533,Land,2015,98000,Gasoline,6-Speed A/T,White,Beige,0,1,240.000000,2.0,4.000000,Gasoline
1,188534,Land,2020,9142,Hybrid,8-Speed A/T,Silver,Black,0,1,395.000000,3.0,6.000000,Gasoline
2,188535,Ford,2022,28121,Gasoline,10-Speed Automatic,White,Ebony,0,1,343.263856,3.5,6.374268,None
3,188536,Audi,2016,61258,Gasoline,Automatic,Silician Yellow,Black,0,1,343.263856,2.0,6.374268,None
4,188537,Audi,2018,59000,Gasoline,A/T,Gray,Black,0,1,252.000000,2.0,4.000000,Gasoline


In [29]:
print(df.isna().sum())
print('---------------')
print(df_test.isna().sum())

id                    0
brand                 0
model_year            0
milage                0
fuel_type             0
transmission          0
ext_col               0
int_col               0
accident              0
clean_title           0
price                 0
Horsepower            0
Displacement       6770
Cylinder Count        0
Fuel Type         31546
dtype: int64
---------------
id                    0
brand                 0
model_year            0
milage                0
fuel_type             0
transmission          0
ext_col               0
int_col               0
accident              0
clean_title           0
Horsepower            0
Displacement       4438
Cylinder Count        0
Fuel Type         21015
dtype: int64


# **Prepare the features for the models**

In [44]:
# Encoding categorical variables
categorical_columns = ['brand', 'fuel_type', 'ext_col', 'int_col','model_year']
le = LabelEncoder()

for col in categorical_columns:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])
        df_test[col] = le.transform(df_test[col])

In [46]:
# scaler = StandardScaler()
scaler = MinMaxScaler()
df[['Horsepower', 'Displacement', 'Cylinder Count']] = scaler.fit_transform(df[['Horsepower', 'Displacement', 'Cylinder Count']])
df_test[['Horsepower', 'Displacement', 'Cylinder Count']] = scaler.transform(df_test[['Horsepower', 'Displacement', 'Cylinder Count']])

In [47]:
df.drop('Fuel Type',axis=1,inplace=True)
df_test.drop('Fuel Type',axis=1,inplace=True)

In [48]:
X = df.drop(columns=['price']) 
y = df['price']

In [49]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

**Linear Regression**

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [51]:
y_pred_lr = lr.predict(X_valid)
y_pred_lr

array([80204.92335142, 32217.64978035, 34234.93304537, ...,
       31818.99644108,  9346.06866586, 48495.0750442 ])

In [52]:
mean_squared_error(y_valid, y_pred_lr, squared=False)

70208.51259503106

In [54]:
y_pred_test_lr = lr.predict(df_test)

In [55]:
# create file to submit linear regression model
# submission_lr = pd.DataFrame({
#     'id': df_test['id'],  
#     'price': y_pred_test_lr
# })

# submission_lr.to_csv('submission_lr.csv', index=False)
# print("Submission file created!")

**Random Forest Regression**

In [56]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [57]:
y_pred_rf = rf.predict(X_valid)
y_pred_rf

array([37022.76, 14076.91, 32068.22, ..., 18665.67,  8433.66, 52980.42])

In [58]:
rmse_rf = mean_squared_error(y_valid, y_pred_rf, squared=False)
rmse_rf

73075.25311703417

In [59]:
y_pred_rf = rf.predict(df_test)

In [ ]:
# create file to submit
# submission_rf = pd.DataFrame({
#     'id': df_test['id'],  
#     'price': y_pred_rf
# })

# submission_rf.to_csv('submission_rf.csv', index=False)
# print("Submission file created!")

**XGBoost**

In [60]:
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb

# XGBoost parameters (you can further tune these)
params = {'n_estimators': 387, 
          'max_depth': 6, 
          'learning_rate': 0.0221955891150421, 
          'subsample': 0.956049799110561, 
          'colsample_bytree': 0.6020390043948215, 
          'gamma': 5.356447556257337e-08, 
          'min_child_weight': 6, 
          'reg_alpha': 1.282064019440197e-05, 
          'reg_lambda': 5.1415598885152124e-05,
          'loss_function': 'RMSE'
         }

xgbm = XGBRegressor(**params)

In [61]:
xgbm.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6020390043948215, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=5.356447556257337e-08,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0221955891150421,
             loss_function='RMSE', max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=6, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=387,
             n_jobs=None, num_parallel_tree=None, ...)

In [62]:
y_pred_xgb = xgbm.predict(X_valid)
rmse_xgb = mean_squared_error(y_valid, y_pred_xgb, squared=False)
rmse_xgb

68147.4609054387

In [63]:
y_pred_xgb_test = xgbm.predict(df_test)

submission_xgb = pd.DataFrame({
    'id': df_test['id'],  
    'price': y_pred_xgb_test
})

submission_xgb.to_csv('submission_xgb.csv', index=False)
print("Submission file created!")

Submission file created!


**LGBM model**

In [64]:
# subsample (default=1.) – Subsample ratio of the training instance.
'''
reg_alpha (default=0) – L1 regularization term on weights.

reg_lambda (default=0) – L2 regularization term on weights.

num_leaves (default=31) – Maximum tree leaves for base learners.
'''

lgbm_params = {
    'subsample': 0.95, 
    'reg_lambda': 0.005623413251903491, 
    'reg_alpha': 1.0, 
    'num_leaves': 570, 
    'n_estimators': 550, 
    'min_data_in_leaf': 135, 
    'min_child_weight': 0.02, 
    'max_depth': 13, 
    'learning_rate': 0.015, 
    'feature_fraction': 0.85, 
    'colsample_bytree': 0.9, 
    'cat_smooth': 50, 
    'bagging_freq': 9, 
    'bagging_fraction': 0.85
}
lgbm_model = lgb.LGBMRegressor(**lgbm_params, random_state=42, verbose=-1)
lgbm_model.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.85, bagging_freq=9, cat_smooth=50,
              colsample_bytree=0.9, feature_fraction=0.85, learning_rate=0.015,
              max_depth=13, min_child_weight=0.02, min_data_in_leaf=135,
              n_estimators=550, num_leaves=570, random_state=42, reg_alpha=1.0,
              reg_lambda=0.005623413251903491, subsample=0.95, verbose=-1)

**CatBoost**

In [65]:
from catboost import CatBoostRegressor
catb_params = {
    'iterations': 1500, 
    'learning_rate': 0.010011982648647914, 
    'depth': 10, 
    'l2_leaf_reg': 9.557443197213443, 
    'bootstrap_type': 'MVS'
}
cat_features = [X_train.columns.get_loc(col) for col in categorical_columns]
catb_model = CatBoostRegressor(**catb_params, cat_features=cat_features, random_state=42, verbose=500)
catb_model.fit(X_train, y_train)

0:	learn: 79745.7525773	total: 346ms	remaining: 8m 38s
500:	learn: 73064.7711039	total: 1m 56s	remaining: 3m 51s
1000:	learn: 72583.6777213	total: 3m 24s	remaining: 1m 41s
1499:	learn: 72346.7197646	total: 4m 28s	remaining: 0us


In [66]:
lgbm_preds = lgbm_model.predict(X_valid)
catb_preds = catb_model.predict(X_valid)

In [68]:
ensemble_preds = (lgbm_preds + catb_preds) / 2
rmse_ensemble = mean_squared_error(y_valid, ensemble_preds, squared=False)
print(f'Ensemble RMSE: {rmse_ensemble}')

Ensemble RMSE: 68035.26279572038


In [69]:
X_test = df_test  
lgbm_test_preds = lgbm_model.predict(X_test)
catb_test_preds = catb_model.predict(X_test)

In [70]:
ensemble_test_preds = (lgbm_test_preds + catb_test_preds) / 2

In [71]:
submission = pd.DataFrame({
    'id': df_test['id'],  
    'price': ensemble_test_preds
})

In [72]:
submission.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!
